In [10]:
import csv
import os
import pandas as pd
def open_file(year,language,OG,analyzed):
    # script_dir = os.path.abspath(__file__)  
    # data_dir = os.path.join(script, "Data")
    # print(data_dir)
    # data_dir = os.path.join(os.path.dirname(__file__), "Data")
    data_dir = 'C:/Users/theoh/Documents/FootdSafetyEngagement/Data'
    filename = f"{data_dir}/tweets_data_{language}_{year}"
    if OG:
        filename += "_VO" 
    if OG == False:
        filename += "_VE"
    if analyzed:
        filename += "_sentiment"
    try:
        with open(filename+'.csv', 'r') as file:
            return pd.read_csv(file,sep=',')
            
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {filename}")
    
def get_file_name(year,language,OG):
    data_dir = 'C:/Users/theoh/Documents/FootdSafetyEngagement/Data'
    filename = f"{data_dir}/tweets_data_{language}_{year}"
    if OG:
        filename += "_VO" 
    if OG == False:
        filename += "_VE"
    return filename

In [6]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import os

def sentiment_analyzing(year,language):
    if language == 'en':
        tweets_data = open_file(year, language, None,False)
    else:
        tweets_data = pd.concat([open_file(year, language, True,False),open_file(year, language, False,False)],ignore_index=True)   

    # Charger le modèle et le tokenizer multilingue
    model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Fonction pour analyser le sentiment d'un tweet
    def analyze_sentiment(tweet):
        inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        scores = outputs[0][0].detach().numpy()
        scores = softmax(scores)
        positive_score = scores[2]  # Utiliser l'indice pour le score positif selon le modèle multilingue
        return 1 if positive_score > 0.5 else 0

    # Appliquer l'analyse de sentiment à chaque tweet
    tweets_data['sentiment'] = tweets_data['content'].apply(analyze_sentiment)

    # Sauvegarder les résultats dans un fichier CSV
    def save_to_csv(dataframe, file_path):
        if os.path.exists(file_path):
            dataframe.to_csv(file_path, mode='a', index=False, header=False, encoding='utf-8')
        else:
            dataframe.to_csv(file_path, mode='w', index=False, encoding='utf-8')

    save_to_csv(tweets_data, get_file_name(year,language,None) + '_sentiment.csv')
    


c:\Users\theoh\miniconda3\envs\Stage\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
# sentiment_analyzing(2024,'en')
tweets_data_en_analyzed = open_file(2024,'en',None,True)
print(tweets_data_en_analyzed['sentiment'])

print("Sur les tweets en anglais, ",tweets_data_en_analyzed['sentiment'].sum(),"sont positifs et ",len(tweets_data_en_analyzed)-tweets_data_en_analyzed['sentiment'].sum(),"sont négatifs.")

0      0
1      0
2      0
3      0
4      0
      ..
333    0
334    0
335    0
336    0
337    0
Name: sentiment, Length: 338, dtype: int64
Sur les tweets en anglais,  16 sont positifs et  322 sont négatifs.
